# Прогнозирование выручки от добычи нефти

**Описание проекта**

Вы работаете в добывающей компании «ГлавРосГосНефть». Нужно решить, где бурить новую скважину. 

Шаги для выбора локации обычно такие:
1. В избранном регионе собирают характеристики для скважин: качество нефти и объём её запасов;
2. Строят модель для предсказания объёма запасов в новых скважинах;
3. Выбирают скважины с самыми высокими оценками значений;
4. Определяют регион с максимальной суммарной прибылью отобранных скважин.

Вам предоставлены пробы нефти в трёх регионах. Характеристики для каждой скважины в регионе уже известны. Постройте модель для определения региона, где добыча принесёт наибольшую прибыль. Проанализируйте возможную прибыль и риски техникой `Bootstrap`.

**Описание данных**

Признаки:

1. `id` — уникальный идентификатор скважины;
2. `f0`, `f1`, `f2` — три признака точек (неважно, что они означают, но сами признаки значимы);

Целевой признак:

1. `product` — объём запасов в скважине (тыс. баррелей).

**Дополнительные условия**

1. Для обучения модели подходит только линейная регрессия (остальные — недостаточно предсказуемые).
2. При разведке региона исследуют 500 точек, из которых с помощью машинного обучения выбирают 200 лучших для разработки.
3. Бюджет на разработку скважин в регионе — 10 млрд рублей.
4. При нынешних ценах один баррель сырья приносит 450 рублей дохода. Доход с каждой единицы продукта составляет 450 тыс. рублей, поскольку объём указан в тысячах баррелей.
5. После оценки рисков нужно оставить лишь те регионы, в которых вероятность убытков меньше 2.5%. Среди них выбирают регион с наибольшей средней прибылью.
6. Данные синтетические: детали контрактов и характеристики месторождений не разглашаются.

In [1]:
import pandas as pd
import optuna
import plotly.graph_objects as go

from plotly.subplots import make_subplots
from IPython.display import display
from collections import defaultdict
from ydata_profiling import ProfileReport

from fast_ml import eda

from sklearn.model_selection import train_test_split
from sklearn.utils import resample

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

from sklearn.linear_model import LinearRegression

from sklearn.metrics import (
    accuracy_score, f1_score, auc, roc_curve, roc_auc_score
)

In [2]:
FIG_WIDTH = 10 * 100
FIG_HEIGHT = 5 * 100
RANDOM_SEED = 42
FILE_NAMES = ['geo_data_0', 'geo_data_1', 'geo_data_2']

In [3]:
raw_oil = {}
for file_name in FILE_NAMES:
    try:
        raw_oil[file_name] = pd.read_csv(file_name + '.csv')
    except:
        raw_oil[file_name] = pd.read_csv('/datasets/' + file_name + '.csv')

## Исследовательский анализ данных

Изучим основные зависимости в данных перед тем, как мы будем использовать их в алгоритмах машинного обучения.

Таблица-резюме:

In [4]:
for file_name in FILE_NAMES:
    display(eda.df_info(raw_oil[file_name]))

,data_type,data_type_grp,num_unique_values,sample_unique_values,num_missing,perc_missing
id,object,Categorical,99990,"[txEyH, 2acmU, 409Wp, iJLyR, Xdl7t, wX4Hy, tL6...",0,0.0
f0,float64,Numerical,100000,"[0.7057449842080644, 1.3347112926051892, 1.022...",0,0.0
f1,float64,Numerical,100000,"[-0.4978225001976334, -0.3401642528583136, 0.1...",0,0.0
f2,float64,Numerical,100000,"[1.22116994843607, 4.3650803324282, 1.41992623...",0,0.0
product,float64,Numerical,100000,"[105.28006184349584, 73.03775026515737, 85.265...",0,0.0


,data_type,data_type_grp,num_unique_values,sample_unique_values,num_missing,perc_missing
id,object,Categorical,99996,"[kBEdx, 62mP7, vyE1P, KcrkZ, AHL4O, HHckp, h5U...",0,0.0
f0,float64,Numerical,100000,"[-15.00134818249185, 14.272087811011149, 6.263...",0,0.0
f1,float64,Numerical,100000,"[-8.275999947188001, -3.47508321506002, -5.948...",0,0.0
f2,float64,Numerical,100000,"[-0.0058760136933206, 0.9991827365665829, 5.00...",0,0.0
product,float64,Numerical,12,"[3.179102583207246, 26.95326103153969, 134.766...",0,0.0


,data_type,data_type_grp,num_unique_values,sample_unique_values,num_missing,perc_missing
id,object,Categorical,99996,"[fwXo0, WJtFt, ovLUW, q6cA6, WPMUX, LzZXx, WBH...",0,0.0
f0,float64,Numerical,100000,"[-1.1469870984179529, 0.2627779016539684, 0.19...",0,0.0
f1,float64,Numerical,100000,"[0.9633279217162892, 0.2698389572803021, 0.289...",0,0.0
f2,float64,Numerical,100000,"[-0.8289649221710994, -2.530186515492004, -5.5...",0,0.0
product,float64,Numerical,100000,"[27.75867323073004, 56.06969663239464, 62.8719...",0,0.0


Числовые распределения:

In [5]:
for file_name in FILE_NAMES:
    display(round(raw_oil[file_name].describe().T, 2))

,count,mean,std,min,25%,50%,75%,max
f0,100000.0,0.50,0.87,-1.41,-0.07,0.50,1.07,2.36
f1,100000.0,0.25,0.50,-0.85,-0.20,0.25,0.70,1.34
f2,100000.0,2.50,3.25,-12.09,0.29,2.52,4.72,16.00
product,100000.0,92.50,44.29,0.00,56.50,91.85,128.56,185.36


,count,mean,std,min,25%,50%,75%,max
f0,100000.0,1.14,8.97,-31.61,-6.30,1.15,8.62,29.42
f1,100000.0,-4.80,5.12,-26.36,-8.27,-4.81,-1.33,18.73
f2,100000.0,2.49,1.70,-0.02,1.00,2.01,4.00,5.02
product,100000.0,68.83,45.94,0.00,26.95,57.09,107.81,137.95


,count,mean,std,min,25%,50%,75%,max
f0,100000.0,0.0,1.73,-8.76,-1.16,0.01,1.16,7.24
f1,100000.0,-0.0,1.73,-7.08,-1.17,-0.01,1.16,7.84
f2,100000.0,2.5,3.47,-11.97,0.13,2.48,4.86,16.74
product,100000.0,95.0,44.75,0.00,59.45,94.93,130.60,190.03


И детальный отчет:

In [6]:
for file_name in FILE_NAMES:
    ProfileReport(raw_oil[file_name]).to_widgets()

Render widgets:   0%|          | 0/1 [00:00<?, ?it/s]

Render widgets:   0%|          | 0/1 [00:00<?, ?it/s]

Render widgets:   0%|          | 0/1 [00:00<?, ?it/s]

Ключевые наблюдения из предварительного анализа набора данных:

1. **Качество данных и типы**: Все три набора данных полные и не имеют пропущенных значений в признаках. Это хорошо, поскольку нам не придется заполнять пропуски, и мы можем прямо переходить к анализу и построению модели. Все наборы данных имеют один категориальный признак `id` и четыре числовых признака: `f0`, `f1`, `f2` и `product`. В дальнейшем `id` можно отбросить, потому что с точки зрения модели, нам не нужен уникальный индентификтор скважины.

2. **Уникальные значения**: Количество уникальных значений в `product` удивительно низкое в наборе данных 2: здесь всего 12 уникальных значений по сравнению с 100 000 в двух других наборах данных. Это может указывать на различия в характере данных в наборе данных 2. Возможно, это связано с тем, как данные были собраны.

3. **Распределение данных**: Большинство значений в колонках расперелены нормально, хотя есть несколько выдяляющихся (например, `f0` в первом датасете). Также, в датасете 2 распределение `f2` и `product` сильно отличается от других. Более того, возможно есть какие-то сложные нелинейные зависимости между различными колонками (например, в датасете 1 мы видим два полумесяца для `f0` vs `f1`).

4. **Корреляция**: Корреляция между признаками варьируется в разных наборах данных. В наборе данных 1 заметна отрицательная корреляция между `f0` и `f1`. В наборе данных 2 `f2` показывает очень сильную положительную корреляцию с `product`, что указывает на то, что `f2` может быть значимым предиктором для `product` в этом наборе данных. В наборе данных 3 все корреляции очень слабые, что указывает на менее прямолинейную связь между переменными.

# Обучение ML моделей

Выполним преобразования на этих датасетах - уберем колонки, которые не нужны для моделей: `id`.

И после разделения на выборки, чтобы избежать data leakage - проведем стандартизацию численных признаков.

Наконец, обучим `LogisticRegression` модель для каждого датасета.

In [9]:
dct_oil = defaultdict(dict)
dct_splits = defaultdict(dict)
dct_models = defaultdict(dict)    

In [30]:
# Define the pipeline
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('model', LinearRegression())
])

# Apply transformations
for file_name in FILE_NAMES:
    
    dct_oil[file_name] = raw_oil[file_name].drop('id', axis=1)
    
    ftr_train, ftr_valid, tgt_train, tgt_valid = train_test_split(
        dct_oil[file_name].drop('product', axis=1), dct_oil[file_name]['product'],
        test_size=0.25, random_state=RANDOM_SEED
    )
    
    pipeline.fit(ftr_train, tgt_train)
    
    dct_models[file_name] = pipeline
    
    dct_splits[file_name] = {
        'ftr_train': pd.DataFrame(ftr_train, columns=ftr_train.columns),
        'ftr_valid': pd.DataFrame(ftr_valid, columns=ftr_valid.columns),
        'tgt_train': pd.DataFrame(tgt_train, columns=['product']),
        'tgt_valid': pd.DataFrame(tgt_valid, columns=['product']),
        'tgt_prdct': pd.DataFrame(pipeline.predict(ftr_valid), index=ftr_valid.index, columns=['product'])
    }

In [31]:
dct_models

defaultdict(dict,
            {'geo_data_0': Pipeline(steps=[('scaler', StandardScaler()), ('model', LinearRegression())]),
             'geo_data_1': Pipeline(steps=[('scaler', StandardScaler()), ('model', LinearRegression())]),
             'geo_data_2': Pipeline(steps=[('scaler', StandardScaler()), ('model', LinearRegression())])})

In [32]:
dct_splits[file_name]['tgt_prdct']

,product
75721,98.301916
80184,101.592461
19864,52.449099
76699,109.922127
92991,72.411847
...,...
21271,102.764169
34014,93.431823
81355,64.098390
65720,83.764135
